In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('movies.csv', index_col=0)
rating = pd.read_csv('ratings.csv')
data_genres = data['genres'].str.get_dummies('|')
rating.drop(['timestamp','userId'], axis = 1, inplace=True)
Rate = rating.groupby('movieId').agg(['mean', 'count'])

In [4]:
data = pd.concat([data,Rate], axis = 1)
#data_genres = pd.concat([data_genres, Rate], axis = 1)
data.rename(columns = {('rating', 'count'):'RateCount', ('rating','mean'):'AvgRating'}, inplace = True)
#data_genres.rename(columns = {('rating', 'count'):'RateCount', ('rating','mean'):'AvgRating'}, inplace = True)
#data_genres.fillna(0, inplace=True)
data.fillna(0, inplace = True)

In [5]:
import re
for idx in data.index:
    try:
        data.loc[data.index == idx, 'Year'] =  re.findall(r"\((\d{4})\)",data.title[idx])
    except:
        pass

data['Year'] = pd.to_numeric(data['Year'], errors= 'raise')
#data.loc[data['Year'] == 1, 'Year'] = int(data.Year.mean())

In [6]:
def Top10(X, Y = 0):
    data1 = data_genres[data_genres[X] == 1]
    data2 = data[(data['Year'] >= Y) & (data.RateCount > 30)].sort_values('AvgRating', ascending = False)
    return (pd.concat([data1, data2], axis = 1, join = "inner")).ix[:,'title':'Year'].head(10)

### Recommendation based on Genre and Year
'Action', 'Adventure', 'Animation', 'Children','Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'

In [7]:
Top10('Comedy', 2000)

,title,genres,AvgRating,RateCount,Year
movieId,,,,,
4973,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance,4.096000,125.0,2001.0
78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,4.071429,42.0,2010.0
4011,Snatch (2000),Comedy|Crime|Thriller,4.036842,95.0,2000.0
109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,4.028571,35.0,2014.0
3481,High Fidelity (2000),Comedy|Drama|Romance,3.960227,88.0,2000.0
57669,In Bruges (2008),Comedy|Crime|Drama|Thriller,3.947368,38.0,2008.0
44195,Thank You for Smoking (2006),Comedy|Drama,3.912500,40.0,2006.0
46578,Little Miss Sunshine (2006),Adventure|Comedy|Drama,3.907895,76.0,2006.0
4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,3.884615,130.0,2001.0


In [8]:
def search(movie):
    x = pd.DataFrame([])
    for i in data.title:
        if movie.lower() in i.lower():
            x = x.append(data.loc[data.title == i])   
    return (x)

### Search a movie by name

In [9]:
search('Inception')

,title,genres,AvgRating,RateCount,Year
movieId,,,,,
79132,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,4.04955,111.0,2010.0


In [241]:
#train, test = train_test_split(data_genres, test_size = 0.3)
from sklearn.cluster import KMeans
model = KMeans(n_clusters=7,max_iter=500)
model.fit(data_genres)
predict = model.predict(data_genres)
#model.score(test)

In [243]:
predict.shape

(9125,)

In [235]:
movie_group = data_genres.copy()

In [245]:
data_genres['Group'] = (predict, index = data_genres.index)

SyntaxError: invalid syntax (<ipython-input-245-96b35644a06d>, line 1)